In [ ]:
# system management
import glob
import os
import importlib

# Array
import xarray as xr
import dask
import numpy as np
import pandas as pd

# See time evolution
from tqdm import tqdm

# Define daily 

In [ ]:
data_folder = r"Y:\archive\GreenFjord\2023\DATA_RAW\CL61"
output_folder = r'C:\Users\gezas\kDrive\SIE-Project\CL61\Data_daily'

In [ ]:
filepaths = glob.glob(os.path.join(data_folder, '*.nc'))
timestamps = [pd.to_datetime(path[-18:-3], format='%Y%m%d_%H%M%S') for path in filepaths]

df_data_files = pd.DataFrame({'file_name_path': filepaths}, index=timestamps)
df_data_files.sort_index(inplace=True)

In [ ]:
df_data_files

In [ ]:
first_date = "2023-06-27 00:00:00"
last_date = "2023-06-29 23:59:00"

In [ ]:
selected_dates = pd.date_range(first_date, last_date, freq='D')
selected_dates

In [ ]:
combined_dataset = None

for selected_date in selected_dates:
    selected_rows = df_data_files.loc[selected_date.strftime('%Y%m%d')]
    for row in tqdm(selected_rows.iterrows(), total=selected_rows.shape[0]):
        # Open the dataset for the selected date
        row_array = xr.open_dataset(row[1]['file_name_path'], chunks='auto')
        # Combine the datasets
        if combined_dataset is None:
            combined_dataset = row_array
        else:
            combined_dataset = xr.concat([combined_dataset, row_array], dim='time')

In [ ]:
start_time = "2023-06-28 00:00:00"
end_time = "2023-06-29 00:00:00"
subset = combined_dataset.sel(time=slice(start_time, end_time))
subset

In [ ]:
# Define the output filename
output_filename = f"cl61_{selected_dates[1].strftime('%Y%m%d')}.nc"

# Define the output file path
output_filepath = os.path.join(output_folder, output_filename)

# Save the combined dataset to a single NetCDF file
subset.to_netcdf(output_filepath, engine='netcdf4')

In [ ]:
date_array = xr.open_mfdataset(selected_rows['file_name_path'], chunks={'time':100}, parallel=True)

In [ ]:
# Define the output filename
output_filename = f"cl61_{selected_date.strftime('%Y%m%d')}-test.nc"

# Define the output file path
output_filepath = os.path.join(output_folder, output_filename)

# Save the combined dataset to a single NetCDF file
combined_dataset.to_netcdf(output_filepath)

print(f"Writing to: {output_filepath}")

In [ ]:
for selected_date in tqdm(selected_dates):
    selected_rows = df_data_files.loc[selected_date.strftime('%Y%m%d')]
    print(f'selected date: {selected_date}')
    if len(selected_rows)==0:
        continue
    date_array = xr.open_mfdataset(selected_rows['file_name_path'], chunks='auto')
    output_filename = f"cl61_{selected_date.strftime('%Y%m%d')}"
    output_filepath = os.join(output_folder, output_filename)
    print(f"writing to : {output_filepath}")
    date_array.to_netcdf(output_filepath)
    print('done')